1st step: Loading of the libraries and listing all of the files in the input directory. 

Files you should see are: 

example_test.csv
lectures.csv
example_sample_submission.csv
questions.csv
train.csv
competition.cpython-37m-x86_64-linux-gnu.so
__init__.py

In [ ]:
!pip install datatable==0.11.0 > /dev/null

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import riiideducation # feather dataset 
import matplotlib.pyplot as plt
from datatable import f, by, sum, sort

In [ ]:
 #used to make feather files easier to load.
 #dtypes = {
    # "row_id": "int64",
    #"timestamp": "int64",
    # "user_id": "int32",
    # "content_id": "int16",
    #"content_type_id": "boolean",
    #"task_container_id": "int16",
    #"user_answer": "int8",
    #"answered_correctly": "int8",
    #"prior_question_elapsed_time": "float32", 
    #"prior_question_had_explanation": "boolean"
 #}

 #files = ['train', 'questions', 'lectures', 'example_test', 'example_sample_submission']

 #for file in files:
     #if file=='train':
        # data = pd.read_csv("../input/riiid-test-answer-prediction/{0}.csv".format(file), dtype=dtypes)
     #else:
         #data = pd.read_csv("../input/riiid-test-answer-prediction/{0}.csv".format(file))
     #data.to_feather("{0}.feather".format(file))
     #print("File: {0} - size: {1}".format(file,data.shape))

2nd Step: Read the train.csv file and generate a table. 

In [ ]:
train_dataframe = dt.fread("../input/riiid-test-answer-prediction/train.csv") #.sort_values('timestamp').reset_index(drop = True)
questions_dataframe = dt.fread('../input/riiid-test-answer-prediction/questions.csv')


In [ ]:
#train_dataframe = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',usecols = ['user_id','content_id','answered_correctly','content_type_id'])
#train_dataframe.sort_values('timestamp').reset_index(drop = True)

In [ ]:
questions_dataframe.shape

3rd Step: Print the user_id values deriving from the train.csv file below. 

In [ ]:
print(train_dataframe['user_id'])

Step 4: Return the table with only columns Content_Id and Answered_Correctly is visible


In [ ]:

def prepare_features(col_name):
    df = train_dataframe[train_dataframe.content_type_id==0][[col_name,'answered_correctly']].groupby(col_name).agg(['count','sum'])
    df.columns=['total', 'positive']
    df = df.astype('uint64')
    df['negative'] = df['total']-df['positive']
    return df
    

In [ ]:
questions_dataframe = prepare_features('content_id')
questions_dataframe

In [ ]:
train_dataframe.content_id.unique()

Step 5: Group the train data frame by user id so we can have an idea how well the student is performing in answering the questions correctly. 

In [ ]:
users_dataframe = prepare_features('user_id')
users_dataframe.head()

In [ ]:
train_dataframe = prepare_features('timestamp')
train_dataframe.head()

Step 6: This bar graph basically sums up the positive values array and the negative values array and 
    generates a bar graph. This will help visualize how well students are performing. 

Step 6: Create the model (Naive Bayes) 

In [ ]:
class NaiveBayes:
    def __init__(self, features, threshold=5):
        assert type(features)==dict, 'parameter features is not a dictionary!'
        for f in features.keys():
            assert type(features[f])==pd.core.frame.DataFrame, 'Wrong datatype for {0}. Each entry of the dictionary must contain a pandas DataFrame'.format(f)
            assert list(features[f].columns)==['total', 'positive', 'negative'], 'wrong columns in {0} DataFrame'.format(f)
        self.THRESHOLD = threshold
        self.features = features
        self.prior_probability = {}
        one_feature = list(features.keys())[0]
        self.prior_probability['negative'] = features[one_feature]['negative'].sum()/features[one_feature]['total'].sum()
        self.prior_probability['positive'] = features[one_feature]['positive'].sum()/features[one_feature]['total'].sum()
        
    def predict(self, data):
        assert data.keys()==self.features.keys(), "Keys doesn't match!"
        data_len = len(data[list(data.keys())[0]])
        # pos and neg are the priors for positive and negative classes
        pos = np.array([self.prior_probability['positive'] for _ in range(data_len)])
        neg = np.array([self.prior_probability['negative'] for _ in range(data_len)])
        # multiply the prior probability by the likelihood of each feature
        for d in data.keys():
            feature = pd.DataFrame({'id':data[d]})
            counts=pd.merge(feature,self.features[d],left_on='id',right_index=True,how='left').fillna(0).astype('uint64').values
            # counts.shape == (sample_len,4)
            # counts[:,0]==id ; counts[:,1]==total ; counts[:,2]==positive ; counts[:,3]==negative
            # e.g.: counts == array([[115,46,32,14],[124,10,7,3],[115,46,32,14]],dtype=uint64)
            updatable = np.where(counts[:,1]>self.THRESHOLD)[0]
            # e.g.: updatable == array([True,False,True])
            pos[updatable] *= counts[updatable,2]/counts[updatable,1]
            neg[updatable] *= counts[updatable,3]/counts[updatable,1]
        return pos/(pos+neg)

In [ ]:
naivebayes = NaiveBayes({'questions': questions_dataframe, 'users':users_dataframe})

Step 7: Conduct predictions on the example_test dataset

Step 8: Prepare submission

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_questions = test_df['content_id']
    test_users = test_df['user_id']
    answered_correctly = naivebayes.predict({'questions':test_questions, 'users':test_users})
    test_df['answered_correctly'] = answered_correctly
    env.predict(test_df.loc[test_df['content_type_id']==0,['row_id','answered_correctly']])